# <font color=darkblue> Machine Learning model deployment with Flask framework on Heroku</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on heroku.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import pickle

### 2. Load the dataset

In [2]:
df = pd.read_csv('car_data.csv')

### 3. Check the shape and basic information of the dataset.

In [3]:
print("Shape:", df.shape)
print("Basic Information:")
print(df.info())

Shape: (301, 9)
Basic Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB
None


### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [4]:
df.drop_duplicates(inplace=True)


### 5. Drop the columns which you think redundant for the analysis.

In [5]:
df.drop(['Car_Name', 'Present_Price', 'Owner'], axis=1, inplace=True)


### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [6]:
df['age_of_the_car'] = 2023 - df['Year']
df.drop('Year', axis=1, inplace=True)


### 7. Encode the categorical columns

In [7]:
df_encoded = pd.get_dummies(df, drop_first=True)


### 8. Separate the target and independent features.

In [8]:
X = df_encoded.drop('Selling_Price', axis=1)
y = df_encoded['Selling_Price']

### 9. Split the data into train and test.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(209, 6) (90, 6)
(209,) (90,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [13]:
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

train_score = rf_model.score(X_train, y_train)
test_score = rf_model.score(X_test, y_test)

print("Train R2 Score:", train_score)
print("Test R2 Score:", test_score)


Train R2 Score: 0.9352764718164517
Test R2 Score: 0.5840727669116539


### 11. Create a pickle file with an extension as .pkl

In [11]:
pickle.dump(rf_model, open('model.pkl', 'wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Used Car Price Prediction</title>
</head>
<body>
    <h1>Used Car Price Prediction</h1>
    <form action="{{ url_for('predict') }}" method="POST">
        <label for="year">Year:</label>
        <input type="text" id="year" name="year"><br><br>
        
        <label for="kms">Kilometers Driven:</label>
        <input type="text" id="kms" name="kms"><br><br>
        
        <label for="fuel">Fuel Type:</label>
        <select id="fuel" name="fuel">
            <option value="Petrol">Petrol</option>
            <option value="Diesel">Diesel</option>
            <option value="CNG">CNG</option>
        </select><br><br>
        
        <label for="seller">Seller Type:</label>
        <select id="seller" name="seller">
            <option value="Dealer">Dealer</option>
            <option value="Individual">Individual</option>
        </select><br><br>
        
        <label for="transmission">Transmission:</label>
        <select id="transmission" name="transmission">
            <option value="Manual">Manual</option>
            <option value="Automatic">Automatic</option>
        </select><br><br>
        
        <input type="submit" value="Predict">
    </form>
</body>
</html>


### b) Create app.py file and write the predict function

In [ ]:
from flask import Flask, render_template, request
import pickle

app = Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    fuel_type = request.form.get('fuel')
    kms_driven = float(request.form.get('kms'))
    seller_type = request.form.get('seller')
    transmission = request.form.get('transmission')
    year = 2023 - int(request.form.get('year'))
    
    features = pd.DataFrame([[kms_driven, year, fuel_type, seller_type, transmission]],
                            columns=['Kms_Driven', 'age_of_the_car', 'Fuel_Type_Diesel', 'Seller_Type_Individual', 'Transmission_Manual'])
    
    prediction = model.predict(features)
    
    return render_template('index.html', prediction_text='The predicted selling price of the used car is {:.2f} lakhs.'.format(prediction[0]))

if __name__ == '__main__':
    app.run(debug=True)


### 13. Deploy your app on Heroku. (write commands for deployment)

### 14. Paste the URL of the heroku application below, and while submitting the solution submit this notebook along with the source code.

### Happy Learning :)